# Modeling

Данный ноутбук посвящён вопросам моделинга

Как всегда импортируем библиотеки и прописываем пути к файлам

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from statistics import mean

from tqdm import tqdm
import copy

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier

import faiss
from faiss import write_index

import pickle

from typing import Dict, List, Tuple, Callable

In [2]:
PATH_TRAIN_FAISS = 'data/preprocessing/train_faiss.csv'
PATH_TEST_FAISS = 'data/preprocessing/test_faiss.csv'
PATH_VALID_FAISS = 'data/preprocessing/valid_faiss.csv'

PATH_TFIDFVECTORIZER = 'models/tfidf.pkl'

PATH_TRAIN_CORPUS = 'data/preprocessing/train_corpus.npz'
PATH_TEST_CORPUS = 'data/preprocessing/test_corpus.npz'
PATH_VALID_CORPUS = 'data/preprocessing/valid_corpus.npz'

RANDOM_STATE = 54321

Загрузим датасеты для обучения

In [3]:
df_train = pd.read_csv(PATH_TRAIN_FAISS)
df_test = pd.read_csv(PATH_TEST_FAISS)
df_valid = pd.read_csv(PATH_VALID_FAISS)

Для того открыть векторизованные корпуса создадим функцию, которая прочитает разряженные матрицы

In [4]:
def load_sparse_csr(filename: str):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix((loader['data'],
                                    loader['indices'],
                                    loader['indptr']),
                                   shape=loader['shape'])

Загрузим векторизованные корпуса

In [5]:
corpus_train = load_sparse_csr(PATH_TRAIN_CORPUS)
corpus_test = load_sparse_csr(PATH_TEST_CORPUS)
corpus_valid = load_sparse_csr(PATH_VALID_CORPUS)

## Модель для нахождения k претендентов на кавер.

Для решения задачи группировки треков будем использовать faiss данная библиотека работает немного быстрее классического варианта K-ближайших, а также имеет возможность дальнейшей оптимизации. Чтобы оценивать качество работы данной модели, а также подбирать гиперпарамтеры воспользуемся метрикой **𝑅𝑒𝑐𝑎𝑙𝑙@𝑘** (полнота на k элементах), поскольку для данного этапа нам необходимо сделать модель, которая будет отбирать максимально подходящие треки для данного трека.

Её можно рассчитать по формуле:

$$ Recall@k =  {\sum найденных \space в \space топk \space матчей \over \sum матчей} $$

Созадим функцию для её рассчёта:

In [6]:
def score_recall_k(y_true: pd.Series, y_pred: pd.Series) -> float:
    metrick_list = []
    for i, y_p in enumerate(y_pred):
        # проверим есть ли для данного трека вообще каверы, усли есть, то
        if y_true[i] is not np.nan:
            for y_t in y_true[i].split():
                metrick_list.append(1 if y_t in y_p else 0)
    return mean(metrick_list)    

Создадим словарь с track_id тренировочного датасета, поскольку фаис будет возвращать индекс строки, а проверять нам потребуется track_id

In [7]:
id_base_dict = dict(df_train['track_id'])

Получим цели обучения (списки с track_id каверов)

In [8]:
y_train = df_train['cover_list']
y_test = df_test['cover_list']
y_valid = df_valid['cover_list']

Напишем функцию для тренировки faiss

In [9]:
def faiss_fit(corpus_train: scipy.sparse, 
              n_list: int = 1, 
              random_state: int = 54321) -> faiss.IndexIVFFlat:
    x_train = copy.deepcopy(corpus_train)
    d = x_train.shape[1] 
    nb = x_train.shape[0] 
    np.random.seed(random_state) 

    xb = x_train.toarray()

    nlist = n_list
    quantizer = faiss.IndexFlatIP(d)  
    index = faiss.IndexIVFFlat(quantizer, d, nlist)
    assert not index.is_trained
    index.train(xb)
    assert index.is_trained
    index.add(xb)
    
    return index

Создадим функцию для предсказаний

In [10]:
def faiss_predict(index: faiss.IndexIVFFlat, 
                  corpus_test: scipy.sparse, 
                  id_base_dict: Dict, 
                  k: int = 10) -> pd.Series:
    x_test = copy.deepcopy(corpus_test)
    xq_x_test = x_test.toarray()
    
    D, I = index.search(xq_x_test, k)
    predicted_list = []
    distance_list = []
    
    # перебираем все ответы и проверяем, чтобы они не были равны 1 и не равнялись
    for i, candidates in enumerate(I):
        cand_list = []
        dist_list = []
        for j, candidate in enumerate(candidates):
            if candidate != -1 and id_base_dict[candidate] != id_base_dict[i]:
                cand_list.append(id_base_dict[candidate])
                dist_list.append(D[i][j])
                
        predicted_list.append(cand_list)
        distance_list.append(dist_list)      

    return pd.Series(predicted_list), pd.Series(distance_list)

Обучим индекс.

In [11]:
f_index = faiss_fit(corpus_train, random_state=RANDOM_STATE)

Проверим качество модели на трейне

In [12]:
y_train_pred, dist_train_pred= faiss_predict(f_index, corpus_train, id_base_dict, 10)
score_recall_k(y_train, y_train_pred)

0.5926640926640927

Проверим качество модели на тесте

In [13]:
y_test_pred, dist_test_pred = faiss_predict(f_index, corpus_test, id_base_dict)
score_recall_k(y_test, y_test_pred)

0.6379310344827587

Выведем для проверки ответ с растояниями для тренировочных данных и ответ с прогнозами

In [14]:
dist_train_pred

0       [1.0000001, 1.0000001, 1.0000001, 1.0000001, 1...
1       [0.09070368, 1.0, 1.0, 1.0, 1.0, 1.0, 1.610105...
2       [1.0, 1.0, 1.0, 1.0, 1.0, 1.258445, 1.2794225,...
3       [0.0, 0.99999994, 0.99999994, 0.99999994, 0.99...
4       [0.7140198, 0.8448281, 0.9239682, 0.9946976, 0...
                              ...                        
2079    [1.0, 1.0, 1.0, 1.0, 1.0, 1.6491057, 1.7004969...
2080    [1.0, 1.0, 1.0, 1.0, 1.0, 1.4479406, 1.4942803...
2081    [1.0, 1.0, 1.0, 1.0, 1.0, 1.2022555, 1.3501427...
2082    [0.99999994, 0.99999994, 0.99999994, 0.9999999...
2083    [1.0000001, 1.0000001, 1.0000001, 1.0000001, 1...
Length: 2084, dtype: object

In [15]:
y_train_pred

0       [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
1       [6c3b156b42de14fd1e222304d14ee50d, b6840c6d29f...
2       [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
3       [ad368cb71bf8aa98e9629bf2a65fc446, b6840c6d29f...
4       [3551cfbcef2af752fe8642e564ae4b5c, d724c8db511...
                              ...                        
2079    [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
2080    [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
2081    [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
2082    [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
2083    [b6840c6d29fadc71fa2d129dadbd066a, 991dd9b0990...
Length: 2084, dtype: object

Немного смущает 0 в 3 строчке, проверим эту строку в тренировочном датасете

In [16]:
df_train.loc[3:3]

,original_track_id,track_id,track_remake_type,translate_text,dttm,title,language,isrc,genres,duration,cover_list,lemm_text
3,43f4ccc81208af9ed8b646ddb3c89a31,43f4ccc81208af9ed8b646ddb3c89a31,ORIGINAL,"Every centimeter, every edge of the soul and b...",2021-03-26 17:17:51,Ты так красива,RU,RUA1D2111316,['POP'],173380.0,NaN,every centimeter every edge of the soul and bo...


Согласно разметке это оригинал и у него нет каверов. Выведем трек с 0 расстоянием из прогноза.

In [17]:
df_train[df_train['track_id']=='ad368cb71bf8aa98e9629bf2a65fc446']

,original_track_id,track_id,track_remake_type,translate_text,dttm,title,language,isrc,genres,duration,cover_list,lemm_text
1885,ad368cb71bf8aa98e9629bf2a65fc446,ad368cb71bf8aa98e9629bf2a65fc446,ORIGINAL,"Every centimeter, every edge of the soul and b...",2022-10-25 09:41:45,Ты так красива,RU,RUA1D2293955,"['POP', 'RUSPOP']",173380.0,NaN,every centimeter every edge of the soul and bo...


Как видим это одна и та же песня, но у них почему то разный track_id и isrc, Но обе почему-то имеют метку оригинала (возможно имеется проблема с разметкой датасета и это нужно будет проверить.

## Модель принимающая решения является ли пара треков каверами

После подбора наиболее подходящих треков создадим новый датасет и поверхнего обучим модель, которая будет решать является ли эта пара треков каверами / оригиналами. Для начала создадим датасет на основании предыдущего этапа моделирования. Рассмотрим следующие варианты создания датасета:

1. Добавления столбцов для обучения для каждого образца и добавления растояния между образцами.
2. Проверка на равнество для категориальных признаков, а для  количественных как в первом варианте.
3. Отношение для количественных признаков, а остальное как в первом варианте.
4. Объединение 2 и 3 варианта.

Выделим нужные категориальные и числовые столбцы

In [18]:
cat_features = ['language', 'title']
num_features = ['duration']

Напишем функцию для переименования столбцов, так как нам потребуется разделять признаки для одного и для второго образца

In [19]:
def rename_columns(df: pd.DataFrame, ind: int) -> pd.DataFrame:
    df = df.copy(deep = True)
    ind = '_' + str(ind)
    columns_dict = dict()
    for column in df.columns:
        columns_dict[column] = column + ind
        
    df = df.rename(columns = columns_dict)
    return df

Напишем  функцию для создания строки датасета по 2 заданным track_id

In [46]:
def get_row_to_df_v1(df: pd.DataFrame, 
                     id_1: str, 
                     id_2: str, 
                     dist: float, 
                     cat_features: List[str] = [], 
                     num_features: List[str] = [], 
                     is_train: bool = True) -> pd.DataFrame:
    all_features = num_features + cat_features
    # получаем нудные столбцы
    df_1 = df.loc[df['track_id'] == id_1, all_features].reset_index(drop=True)
    df_2 = df.loc[df['track_id'] == id_2, all_features].reset_index(drop=True)
    # переименовываем столбцы
    df_1 = rename_columns(df_1, 1)
    df_2 = rename_columns(df_2, 2)
    # объединяем
    new_df = pd.concat([df_1, df_2], axis=1)
    # добавляем расстояние
    new_df['dist'] = dist
    # если тренировочный датасет, то смотрим не входит ли 1 id в список каверов второго и наоборот
    #предварительно проверяя на nan
    target_value = ((~df.loc[df['track_id']==id_1, 'cover_list'].isna().any() and
                   id_2 in df.loc[df['track_id']==id_1, 'cover_list'].values[0]) or 
                   (~df.loc[df['track_id']==id_2, 'cover_list'].isna().any() and
                   id_1 in df.loc[df['track_id']==id_2, 'cover_list'].values[0]))
    new_df['target'] = int(target_value)
    return new_df

Создадим функцию для генерации всего датсета, при этом сделаем так, что бы она на вход принимала функцию генерации признаков, это позволит в дальнейшем переиспользовать её.

In [47]:
def get_df(df: pd.DataFrame, 
           series_pred: pd.Series, 
           series_dist: pd.Series, 
           function: Callable, 
           cat_features: List[str] = [], 
           num_features: List[str] = [],
           is_train: bool = True) -> pd.DataFrame:
    new_df = pd.DataFrame()
    for i in tqdm(range(series_pred.shape[0])):
        list_pred = series_pred[i]
        list_dist = series_dist[i]
        for j in range(len(list_pred)):
            df_match = function(df, 
                                df.loc[i, 'track_id'], 
                                list_pred[j],
                                list_dist[j],
                                cat_features,
                                num_features)
            new_df = pd.concat([new_df, df_match])
    return new_df

Сгенерируем датасет

In [48]:
df_cat_train = get_df(df_train, y_train_pred, dist_train_pred, get_row_to_df_v1, cat_features, num_features)

100%|██████████████████████████████████████████████████████████████████████████████| 2084/2084 [01:05<00:00, 31.90it/s]


In [49]:
df_cat_train.head()

,duration_1,language_1,title_1,duration_2,language_2,title_2,dist,target
0,188240.0,RU,Стань,163290.0,EN,Over You,1.0,0
0,188240.0,RU,Стань,136640.0,EN,Fenomen,1.0,0
0,188240.0,RU,Стань,105880.0,EN,Enigma,1.0,0
0,188240.0,RU,Стань,108720.0,EN,escape,1.0,0
0,188240.0,RU,Стань,231500.0,EN,Baby mama,1.0,0


Поскольку названия столбцов в датасете менялись, то нужно выбрать новые списки категориальных и числовых признаков

In [50]:
def get_features_list(df: pd.DataFrame,
                      cat_features: List[str] = [], 
                      num_features: List[str] = []) -> Tuple[List, List]:
    columns = df.columns
    new_cat_features = []
    new_num_features = []
    for column in columns:
        if column.split('_')[0] in cat_features:
            new_cat_features.append(column)
        if column.split('_')[0] in num_features:
            new_num_features.append(column)
    return new_cat_features, new_num_features

In [51]:
new_cat_features, new_num_features = get_features_list(df_cat_train, cat_features, num_features)

Посмотрим как распределён целевой признак.

In [52]:
df_cat_train['target'].mean()

0.049104286628278954

Выделим признаки и цели обучения

In [53]:
X_train = df_cat_train.drop('target', axis=1)
y_train = df_cat_train['target']

Как видим у нас всего около 5% матчей, поэтому при обучении необходимо указать модели на дисбаланс классов. Как отмечалось ранее для обучения будем использовать CatBoostClassifier. Поскольку у нас задача свелась к задачи классификации, то в качестве метрики будем использовать f1 меру.

In [62]:
cat_model = CatBoostClassifier(random_state=RANDOM_STATE, 
                               cat_features=new_cat_features,
                               verbose=False)

Определять качество модели будем на кроссвалидации, поскольку у нас крайне мало целевых значений, то для разделения на фолды будем использовать StratifiedKFold

In [73]:
skf = StratifiedKFold(n_splits=4)
folds = skf.split(X_train, y_train)

In [74]:
%%time
cvs = cross_val_score(cat_model, X_train, y_train, cv=folds, scoring='f1')
print(f'Метрика f1 на кроссвалидации {cvs}')
print(f'Среднее значение f1 на всём датасете {mean(cvs)}')

Метрика f1 на кроссвалидации [0.63063063 0.79254079 0.22393822 0.3115942 ]
Среднее значение f1 на всём датасете 0.4896759625020494
CPU times: total: 11min 25s
Wall time: 1min 31s


Проверим нашу гипотезу, что в данных сильный дисбаланс классов и передача весов, поможет улучшить качество модели.

In [79]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [81]:
cat_model = CatBoostClassifier(random_state=RANDOM_STATE, 
                               cat_features=new_cat_features,
                               verbose=False,
                               class_weights=class_weights)

In [82]:
%%time
skf = StratifiedKFold(n_splits=4)
folds = skf.split(X_train, y_train)

cvs = cross_val_score(cat_model, X_train, y_train, cv=folds, scoring='f1')
print(f'Метрика f1 на кроссвалидации {cvs}')
print(f'Среднее значение f1 на всём датасете {mean(cvs)}')

Метрика f1 на кроссвалидации [0.68501529 0.87784679 0.56615385 0.32374101]
Среднее значение f1 на всём датасете 0.6131892336895594
CPU times: total: 10min 12s
Wall time: 1min 28s


Действительно, сбалансированне позволяют улучшить качество модели.